In [1]:
import geopack
from geopack import geopack

Load IGRF coefficients ...


In [1]:
import geopack
import pyspedas
import numpy as np
import pytplot
from datetime import datetime, timedelta, timezone
from pytplot import tplot, data_quants , store_data, options, cdf_to_tplot
from geopack import geopack

def compara_beq(time):
    timestamp = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')

    # 年、月、日、時、分を取得し、文字列に変換
    year = str(timestamp.year)
    month = str(timestamp.month).zfill(2)  # ゼロ埋め
    day = str(timestamp.day).zfill(2)  # ゼロ埋め
    hour = str(timestamp.hour).zfill(2)  # ゼロ埋め
    minute = str(timestamp.minute).zfill(2)  # ゼロ埋め

    timestamp_next_day = timestamp + timedelta(days=1)
    nextday = str(timestamp_next_day.day).zfill(2) 

    ### if you want to import the data into your repository, use this code. ###
    #tr = ['{}-{}-{}'.format(year, month, day), '{}-{}-{}'.format(year, month, nextday)]

    ### if you want to import the data into your repository, change this code and variable name of 'pos_gsm' ###
    #cdf_to_tplot('/home/kooki/Documents/duct_analysis/basic/2024Mayevent/erg_data/satellite/erg/orb/def/{}/erg_orb_l2_{}{}{}_v04.cdf'.format(year, year, month, day))
    cdf_to_tplot('/home/kooki/Documents/duct_analysis/basic/2024May10/erg_orb_l2_20240511_v04.cdf')

    datetime_array = pytplot.data_quants['pos_gsm'].coords['time'].values
    # Target datetime
    target_datetime = np.datetime64(time)
    index = np.where(datetime_array == target_datetime)[0][0]
    data = pytplot.data_quants['pos_gsm'].values[index]

    dt = datetime(int(year), int(month), int(day), int(hour), int(minute), 0, tzinfo=timezone.utc)

    # UNIXエポックからの経過時間を計算します
    epoch = datetime(1970, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
    elapsed_time = dt - epoch

    # 経過時間を秒単位で取得します
    ut = elapsed_time.total_seconds()
    ps = geopack.recalc(ut)

    # 南北両方向にトレースして結合
    x,y,z, xx, yy, zz = geopack.trace(data[0], data[1], data[2], 1)
    x_n, y_n, z_n, xx_n, yy_n, zz_n = geopack.trace(data[0], data[1], data[2], -1)
    combined_xx = np.concatenate(( np.flip(xx_n), xx))
    combined_yy = np.concatenate(( np.flip(yy_n), yy))
    combined_zz = np.concatenate(( np.flip(zz_n), zz))

    # トレースした座標での磁場を計算
    bb = np.zeros(len(combined_xx))
    for i in range(len(combined_xx)):
        bx, by, bz = geopack.t89.t89(2, ps, combined_xx[i], combined_yy[i], combined_zz[i])
        bx0, by0, bz0 = geopack.dip(combined_xx[i], combined_yy[i], combined_zz[i])
        bb[i] = ( np.sqrt((bx + bx0)**2 + (by + by0)**2 + (bz + bz0)**2) )

    # 最小値を取得
    min_b_t = bb[np.argmin(bb)]

    ### if you wnat to compare with erg orb l3 data, use this code. ###
    """ #from pyspedas.erg import orb
    #orb(tr, level='l3')        erg_orb_l3_
    cdf_to_tplot('./erg_data/satellite/erg/orb/l3/opq/2018/02/erg_orb_l3_t89_20180222_v02.cdf')
    datetime_array = pytplot.data_quants['pos_beq_t89'].coords['time'].values
    # Target datetime
    target_datetime = np.datetime64(time)
    index = np.where(datetime_array == target_datetime)[0][0]
    b_eq = pytplot.data_quants['pos_beq_t89'].values[index] """

    return min_b_t  #, b_eq

Load IGRF coefficients ...


In [2]:
time = '2024-05-12 13:30:00'
min_b_t = compara_beq(time)

TypeError: 'CDFInfo' object is not subscriptable

In [5]:
year

NameError: name 'year' is not defined

In [9]:
cdf_to_tplot('/home/kooki/Documents/duct_analysis/basic/2024May10/erg_pwe_hfa_l2_spec_monit_20240511_v01_02.cdf')

TypeError: 'CDFInfo' object is not subscriptable